[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/eirasf/GCED-AA2/blob/main/lab1/p1-2.ipynb)

# Práctica 1: Redes neuronales desde cero - Parte 2 - PyTorch

En esta segunda parte de la práctica vamos a utilizar PyTorch para implementar y entrenar la misma red neuronal que desarrollamos con Numpy en la parte 1.

Necesitaremos, por tanto, la librería `torch` además de las ya utilizadas `numpy`, `pandas` y `seaborn`.

In [ ]:
import torch
import numpy as np
import seaborn as sns
import pandas as pd

# Establecemos una semilla aleatoria para que los resultados sean reproducibles en distintas ejecuciones
np.random.seed(1234567)

Cargaremos el conjunto de datos `titanic`, tal como hicimos en la parte 1 de la práctica pero transformando `X` e `y` en tensores de `torch`. Obtendremos dos tensores (`vectores_x` y `etiquetas`) que serán los que utilizaremos posteriormente.

In [ ]:
import seaborn as sns
from sklearn.preprocessing import StandardScaler

def load_titanic():
    # Cargamos el dataset Titanic desde seaborn
    df = sns.load_dataset('titanic')

    # 1️⃣ Selección de variables relevantes y limpieza
    # Columnas que vamos a usar
    cols = ['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked', 'alone']
    df = df[cols].copy()

    # Eliminamos filas con valores faltantes
    df = df.dropna(subset=['age', 'embarked', 'fare'])

    # 2️⃣ Separar etiquetas y características
    y = df['survived'].to_numpy().astype(np.float32)        # etiquetas como float
    X = df.drop(columns=['survived'])

    # 3️⃣ One-hot encoding para todas las variables categóricas
    categorical_cols = ['pclass', 'sex', 'embarked', 'alone']
    X_encoded = pd.get_dummies(X, columns=categorical_cols, drop_first=True)  # drop_first=True evita multicolinealidad

    # 4️⃣ Variables numéricas
    numeric_cols = ['age', 'sibsp', 'parch', 'fare']
    X_numeric = X_encoded[numeric_cols + [c for c in X_encoded.columns if c not in numeric_cols]]
    scaler = StandardScaler()
    X_numeric[numeric_cols] = scaler.fit_transform(X_numeric[numeric_cols])

    # 5️⃣ Convertir a numpy arrays
    X_np = X_numeric.to_numpy().astype(np.float32)
    y_np = y.reshape(-1, 1).astype(np.float32)  # reshape para que sea (n_samples,1)

    return X_np, y_np

vectores_x, etiquetas = load_titanic()
vectores_x = torch.from_numpy(vectores_x)
etiquetas = torch.from_numpy(etiquetas)

## Declaración del modelo

En primer lugar, debemos crear en TensorFlow el grafo de operaciones que representa nuestro modelo. Para ello:
 1. Creamos las variables que TF optimizará, es decir, los parámetros del modelo.
 1. Creamos el grafo de operaciones que producen la predicción a partir de la entrada y las variables. En este caso utilizaremos funciones que relacionen variables de TF con tensores que contendrán datos utilizando operaciones de TF.

In [ ]:
# Variables auxiliares
tamano_entrada = vectores_x.shape[1]
h0_size = 5
h1_size = 3

# CREACIÓN DE LAS VARIABLES
# TODO - Completa las dimensiones de las matrices
W0 = torch.tensor(np.random.randn(h0_size, tamano_entrada), dtype=torch.float32, requires_grad=True)
b0 = torch.tensor(np.random.randn(1, h0_size), dtype=torch.float32, requires_grad=True)
W1 = torch.tensor(np.random.randn(h1_size, h0_size), dtype=torch.float32, requires_grad=True)
b1 = torch.tensor(np.random.randn(1, h1_size), dtype=torch.float32, requires_grad=True)
W2 = torch.tensor(np.random.randn(1, h1_size), dtype=torch.float32, requires_grad=True)
b2 = torch.tensor(np.random.randn(1, 1), dtype=torch.float32, requires_grad=True)

# Guardamos todas las variables en una lista para posteriormente acceder a ellas fácilmente
VARIABLES = [W0, b0, W1, b1, W2, b2]


# CREACIÓN DEL GRAFO DE OPERACIONES
def capa_sigmoide(x, W, b):
    # TODO - Completa con funciones de tensorflow el cálculo de la salida de una capa en la siguiente línea
    return 

def predice(x):
    # TODO - Completa las siguientes líneas
    h0 = 
    h1 = 
    y = 
    return y

# Verificación
x_test = np.random.randn(1,tamano_entrada)
y_pred = predice(x_test) 
print(y_pred)
np.testing.assert_almost_equal(0.494716, y_pred.detach().numpy(), err_msg='Revisa tu implementación')

## Entrenamiento del modelo
El modelo declarado ya se puede utilizar para hacer predicciones pasándole a la función `predice` un tensor con datos (tal como se ha hecho en el apartado de verificación de la celda anterior). Sin embargo, como vimos en la parte 1, este modelo no está ajustado a los datos de entrada, por lo que producirá malas predicciones.

Debemos encontrar un conjunto de valores para los parámetros ($\mathbf{W}_2$, $b_2$, $\mathbf{W}_1$, $\mathbf{b}_1$, $\mathbf{W}_0$ y $\mathbf{b}_0$) que minimicen la función de coste. TensorFlow nos ayuda a optimizar este proceso.

TensorFlow permite configurar el proceso de optimización, por lo que deberemos indicarle:
 1. Qué función de pérdida queremos. En nuestro caso habíamos elegido la entropía cruzada binaria.
 1. Qué método de optimización utilizar. Como en la parte 1, utilizaremos descenso de gradiente.
 
Por el momento crearemos sendas variables para almacenar ambas configuraciones. Al estar organizado de esta manera, utilizar una función de pérdida distinta o un algoritmo de optimización diferente será tan sencillo como cambiar estas variables.

In [ ]:
import torch.nn as nn
import torch.optim as optim

# Función de pérdida (Binary Cross Entropy)
fn_perdida = nn.BCELoss()  # espera que las salidas sean entre 0 y 1

# Optimizador SGD con learning rate 0.1
# VARIABLES es la lista de tensores con requires_grad=True
optimizador = optim.SGD(VARIABLES, lr=0.1)
# optimizador = optim.Adam(VARIABLES, lr=0.1)

### El bucle de entrenamiento

El bucle de entrenamiento será análogo al utilizado en la parte 1. Consistirá en ejecutar un número preestablecido (`NUM_EPOCHS`) de pasos de entrenamiento. En cada paso haremos lo siguiente:
 1. Tomar los datos de entrada y calcular las predicciones que hace el modelo en su estado actual
 1. Calcular el coste (la media de las pérdidas de cada predicción)
 1. Utilizar el valor de coste para actualizar cada variable en dirección de su gradiente

Crearemos una función `paso_entrenamiento` que realice este trabajo. PyTorch se ocupará de calcular los gradientes y realizar las actualizaciones de las variables. Para calcular los gradientes, debemos llamar a la función `backward` sobre el tensor que contiene el valor que queremos optimizar. Una vez la hayamos llamado, podemos pedir al optimizador que dé un `step` en la dirección de descenso del gradiente.



In [ ]:
num_elems = vectores_x.shape[0]

def paso_entrenamiento(x, y):
    # Aseguramos que los gradientes se inicialicen a cero
    optimizador.zero_grad()
    
    # TODO - Completa la siguiente línea para que calcule las predicciones
    y_pred = 
    
    # Cálculo de la pérdida utilizando la función que hemos escogido anteriormente
    perdida = fn_perdida(y, y_pred)

    # Esto computa el gradiente de la pérdida con respecto a todos los tensores que tienen requires_grad = True
    perdida.backward()
    
    # Realizar la actualización de las variables solo requiere esta llamada
    optimizador.step()
    
    # Tasa de acierto (accuracy)
    fallos = torch.abs(y.reshape(-1,1) - y_pred)
    tasa_acierto = torch.sum(1 - fallos)
    
    # Devolvemos estos dos valores para poder mostrarlos por pantalla cuando estimemos conveniente
    return (perdida, tasa_acierto)

# PROCESO DE ENTRENAMIENTO
num_epochs = 10000
for epoch in range(num_epochs):    
    perdida, tasa_error = paso_entrenamiento(vectores_x, etiquetas)
    
    if epoch % 100 == 99:
        print("Epoch:", epoch, 'Pérdida:', perdida.numpy(), 'Tasa de acierto:', tasa_error.numpy()/num_elems)


El uso de PyTorch nos ha permitido abstraernos de los detalles de implementación y del cálculo de derivadas para centrarnos en la arquitectura de nuestro modelo.